# Clase 36

Para una mejor visualización entrar al siguiente [link](https://nbviewer.jupyter.org/github/racsosabe/Miscelanea/blob/master/UPC/Clase%2036%20-%20Strings%20IV.ipynb)

**Nota**: El contenido de la clase 35 estaba en el notebook de la clase 34.

# Requisitos Previos

* Matemática Básica
* Matemática Discreta

# Suffix Array

El *suffix array* es una estructura de datos que nos permite ordenar los sufijos de un string $s$ mediante un arreglo $a$, de manera que el elemento $a_{i} = p$ representa al sufijo de $s$ que empieza en la posición $p$.

## Comparando strings

Recordemos el problema de comparar dos subsstrings $a$ y $b$ de la misma longitud $m$, anteriormente usamos hashing para poder tener una complejidad de $O(1)$ con un preprocesamiento de $O(n)$ si la cadena de la cual son substring tiene longitud $n$. Sin embargo, esta comparación tiene una probabilidad de fallo y una constante pesada por la aritmética modular.

En primer lugar, deberemos realizar una observación:

**Observación 1:** La cantidad de substrings de $s$ ($|s| = n$) con longitud $m$ es $n - m + 1 = O(n)$

Esto quiere decir que si encontramos una forma de ordenar esos $O(n)$ valores, podemos mapear dichos substrings a la cantidad de substrings menores lexicográficamente y así poder comparar en $O(1)$, pues almacenaríamos esta cantidad en un entero.

Claramente no podemos usar un almacenamiento por cada longitud posible porque esto nos llevaría a una complejidad de $O(n^{3}\log{n})$, así que plantearemos una pequeña variación de Sparse Table para obtener los mapeos.

Usaremos un arreglo $ST[N][LOG]$ de manera que $ST[i][k]$ es el mapeo del substring de $s$ que empieza en la posición $i$ y tiene longitud $2^{k}$. Así, podremos armar cada nivel $k$ usando el nivel anterior para todos los $k > 0$, mientras que el nivel $k = 0$ lo construimos de manera directa porque son cadenas de un solo caracter.

1. Un substring de longitud $2^{k}$ con $k > 0$ es la concatenación de dos substrings de longitud $2^{k - 1}$, por lo tanto, este puede ser representado como el par $(m_{1}, m_{2})$, donde $m_{1}$ y $m_{2}$ son los mapeos de los substring que lo componen.

2. Dados los mapeos, es sencillo notar que para comparar dos substring de longitud $2^{k}$ basta con comparar su par $(m_{1}, m_{2})$ lexicográficamente.

Si usamos un `std::sort` para los pares, podremos asignar los mapeos en $O(n\log{n})$ por nivel, y ya que tenemos $O(\log{n})$ niveles, la complejidad total será $O(n\log^{2}{n})$.

```C++
memset(ST, -1, sizeof ST);
vector<int> p(n);
iota(p.begin(), p.end(), 0);
sort(p.begin(), p.end(), [&] (int i, int j){
    return s[i] < s[j];
});
int cnt = 0;
for(int i = 0; i < n; i++){
    if(i > 0 and s[p[i - 1]] != s[p[i]]) cnt += 1;
    ST[p[i]][0] = cnt;
}
for(int d = 1; 1 << d <= n; d++){
    int dis = 1 << (d - 1);
    vector< pair<int, int> > mapeos;
    int limit = 0;
    for(int i = 0; i + 2 * dis - 1 < n; i++){
        mapeos.emplace_back(make_pair(ST[i][d - 1], ST[i + dis][d - 1]));
        limit += 1;
    }
    p.resize(limit);
    iota(p.begin(), p.end(), 0);
    sort(p.begin(), p.end(), [&] (int i, int j){
        return mapeos[i] < mapeos[j];
    });
    int cnt = 0;
    for(int i = 0; i < limit; i++){
        if(i > 0 and mapeos[p[i - 1]] != mapeos[p[i]]) cnt += 1;
        ST[p[i]][d] = cnt;
    }
}
```

Inicializamos con el valor de $-1$ toda la matriz para representar a la cadena vacía en caso de no visitar alguna posición.

Analicemos el resultado de ejecutar el algoritmo sobre la cadena $s = abaaba$:

1. Nivel $k = 0$: Solo hay las cadenas $a$ y $b$, así que a los caracteres $a$ se les asignará $0$ y a los $b$, $1$. 

2. Nivel $k = 1$: Las cadenas son $aa$, $ab$, $ba$, así que tendremos $0$, $1$ y $2$, respectivamente.

3. Nivel $k = 2$: Las cadenas son $aaba$, $abaa$ y $baab$.

Nuestra matriz final será:

$$ ST[N][LOG] = \begin{pmatrix} 0 &1 &0 &0 &1 &0 \\ 1 &2 &0 &1 &2 &-1 \\ 1 &2 &0 &-1 &-1 &-1 \end{pmatrix} $$

### Comparando lexicográficamente de manera naive

Para comparar dos substring lexicográficamente nos basta con ir quitando la máxima potencia de 2 que no exceda a la longitud actual y comparar los resultados lexicográficamente.

Notemos que este orden debe ser el mismo para ambos substring, sino la comparación no será entre substrings de igual longitud y por ende no será válida. 

Entonces, podremos comparar dos strings lexicográficamente en $O(\log{n})$, pues la cantidad de mapeos a obtener es igual a la cantidad de bits prendidos de la longitud.

```C++
vector<int> getMapping(int l, int r){
    vector<int> mapping;
    while(l <= r){
        int k = 31 - __builtin_clz(r - l + 1);
        mapping.emplace_back(ST[l][k]);
        l += 1 << k;
    }
    return mapping;
}

bool lesseq(int l1, int r1, int l2, int r2){
     return getMapping(l1, r1) <= getMapping(l2, r2);
}
```

### Comparando igualdad

Para comparar la igualdad de dos substrings podríamos usar el método anterior; sin embargo, podemos notar que si usamos la query de Sparse Table para funciones idempotentes, esta nos servirá para comparar substrings en $O(1)$. Se puede probar que la siguiente función compara correctamente dos substrings del mismo tamaño.

```C++
bool areequal(int l1, int r1, int l2, int r2){
    int d = r1 - l1 + 1;
    int k = 31 - __builtin_clz(r1 - l1 + 1);
    int dis = 1 << k;
    return make_pair(ST[l1][k], ST[r1 - dis + 1][k]) == make_pair(ST[l2][k], ST[r2 - dis + 1][k]);
}
```

Se puede probar también que podemos extender esta técnica para comparar lexicográficamente en $O(1)$ y ya no en $O(\log{n})$.

## Ordenando los sufijos

Cuando deseamos comparar substrings del mismo tamaño no tendremos problemas si usamos el método anterior; sin embargo, los sufijos de un string $s$ tienen diferentes tamaños, por lo que deberemos pensar alguna alternativa.

- 1ra Opción: Construir siempre los $n$ pares de substring para cada nivel, considerando $-1$ como mapeo de la cadena vacía. Esto nos permite obtener el mapeo de cada sufijo en el último nivel.

- 2da Opción: Considerar los substrings como cíclicos, de esta forma obtendremos siempre substrings de la misma longitud. Para poder ordenar los sufijos de manera correcta, deberemos agregar un caracter separador $\# \not \in \Sigma$ y con un mapeo menor que cualquier caracter de $\Sigma$ al final de $s$ y en cada nivel siempre se mapearán correctamente los substrings.

Ya que la 2da opción es más flexible en términos de aplicación, la tomaremos como referencia para la construcción del Suffix Array.

### Optimizando la complejidad

Hasta ahora tenemos un algoritmo que toma $O(n\log^{2}{n})$ para la construcción del Sparse Table, sin embargo, podemos reciclar memoria para usar solo $O(n)$ de complejidad espacial en vez de $O(n\log{n})$. Precisamente el resultado de $p$ en el último nivel es el suffix array de la cadena.

Notemos que al usar `std::sort` obtenemos una complejidad de $O(n\log{n})$ por nivel, pero es importante notar que los mapeos realizados en cada nivel a partir de $k = 1$ conllevan a tener valores $m_{1}$ y $m_{2}$ menores que $n$. Esta propiedad nos permite usar *Radix Sort* para realizar el ordenamiento de los pares en $O(n)$ y reducir nuestra complejidad a $O(n\log{n})$. 

**Nota:** El primer nivel puede no tener valores menores que $n$, pero en ese caso podemos realizar un `std::sort` una vez y la complejidad no se verá afectada.

**Observación 2:** Si dos sufijos cíclicos $s_{1}$ y $s_{2}$ de longitud $m \geq n$ (donde $n$ es la longitud de la cadena de la cual son sufijos) se comparan, entonces se cumple una de las dos siguientes posibilidades:

1. Si $s_{1}$ y $s_{2}$ coinciden en los primeros $n$ caracteres, entonces los sufijos son iguales.

2. En caso contrario, $s_{1}$ y $s_{2}$ son diferentes y la primera posición en la que difieran estará en los primeros $n$ caracteres.

Gracias a la observación anterior, necesitamos que los sufijos tengan una longitud de al menos $n$ caracteres para poder compararlas de manera adecuada, esto quiere decir que deberemos duplicar el tamaño de las cadenas cíclicas hasta que el tamaño $L$ de todas sea $L \geq n$.

### Primera implementación

En primer lugar, construiremos el primer nivel de mapeo ordenando el string. Dado que solo haremos esto una sola vez, la complejidad no se verá afectada. Luego de esto, construiremos cada mapeo duplicando el tamaño de las cadenas hasta que el tamaño de los substring sea mayor o igual a $n$.

Para poder saber la longitud, mantendremos una variable $len$; por otra parte, para saber cuándo aumentar el contador en el momento de construir el nuevo mapeo es algo análogo al del Sparse Table (Aumenta al encontrar un nuevo par o caracter). Es evidente que antes de realizar el nuevo mapeo, deberemos ejecutar un Radix Sort sobre los pares, pero como son solamente 2 iteraciones, nos basta tener una función que realice Counting Sort sobre una permutación asociada a un vector de valores para llamarla 2 veces.

```C++
void counting_sort(vector<int> &p, vector<int> &values, int n){
	vector<int> cnt(n, 0);
	vector<int> head(n, 0);
	for(int i = 0; i < n; i++){
		cnt[values[p[i]]] += 1;
	}
	for(int i = 1; i < n; i++){
		head[i] = head[i - 1] + cnt[i - 1];
	}
	vector<int> new_p(n);
	for(int i = 0; i < n; i++){
		new_p[head[values[i]]++] = p[i];
	}
	swap(new_p, p);
}

vector<int> suffix_array(){
	vector<int> a(n);
	vector<int> mapeo(n);
	iota(a.begin(), a.end(), 0);
	sort(a.begin(), a.end(), [&] (int i, int j){
		return s[i] < s[j];
	});
	int cnt = 0;
	mapeo[a[0]] = 0;
	for(int i = 1; i < n; i++){
		mapeo[a[i]] = mapeo[a[i - 1]] + (!(s[a[i]] == s[a[i - 1]]));
	}
	int len = 1;
	vector<int> new_mapeo(n);
	while(len < n){
		vector<int> values;
		for(int i = 0; i < n; i++){
			values.emplace_back(mapeo[(i + len) % n]);
		}
		iota(a.begin(), a.end(), 0);
		counting_sort(a, values, n);
		for(int i = 0; i < n; i++){
			values[i] = mapeo[a[i]];
		}
		counting_sort(a, values, n);
		new_mapeo[a[0]] = 0;
		for(int i = 1; i < n; i++){
			if(mapeo[a[i]] != mapeo[a[i - 1]]){
				new_mapeo[a[i]] = new_mapeo[a[i - 1]] + 1;
			}
			else{
				int pre = mapeo[(a[i] + len) % n];
				int cur = mapeo[(a[i - 1] + len) % n];
				new_mapeo[a[i]] = new_mapeo[a[i - 1]] + (pre != cur);
			}
		}
		swap(mapeo, new_mapeo);
		len <<= 1;
	}
	return a;
}
```

Este código tiene un rendimiento aceptable con complejidad $O(n\log{n})$; sin embargo, requiere que implementemos dos funciones y al ser tantas líneas, tomará más tiempo debuggearlo de ser necesario.

### Usemos una sola función y reduzcamos líneas

La idea del Radix Sort es algo importante para mejorar la complejidad, pero es importante notar que al momento en que llegamos a la iteración dentro del `while`, los valores de $a_{i}$ ya estarán ordenados por su mapeo. Esta observación nos ayuda porque podemos cambiar nuestro enfoque:

En vez de considerar el mapeo $(i, i + len)$, podemos considerar el mapeo $(i - len, i)$, de esta forma $a$ nos dará una secuencia de pares que ya estarán ordenados por su segunda componente, por lo que podemos aplicar un counting sort solo 1 vez. Entonces, como es solo 1 counting sort, podemos implementarlo dentro del mismo bucle sin necesidad de llamar a una función extra. Introduciremos el $a$ temporal como la variable $sorted\_by\_second$ para asignar los nuevos pares.

Además, aprovecharemos aún más el valor de $a$, considerando que en la función `counting_sort` creábamos dos vectores $cnt$ y $head$, donde $cnt$ era usado para asignar las posiciones del primer elemento de un determinado valor y esta posición era almacenada en $head$; sin embargo, si iteramos sobre todos los elementos en orden decreciente, entonces podemos realizar la asignación $head[val[i]] = i$ y eventualmente el valor de $head[val[i]]$ almacenará la posición del primer elemento con valor $val[i]$. Dado que el conjunto de las primeras componentes de los pares generados es el mismo que los mapeos de $a$, podemos usar este truco para usar solo un vector extra y omitir la llamada a una segunda función.

```C++
vector<int> suffix_array(){
	vector<int> a(n);
	vector<int> mapeo(n);
	iota(a.begin(), a.end(), 0);
	sort(a.begin(), a.end(), [&] (int i, int j){
		return s[i] < s[j];
	});
	int cnt = 0;
	mapeo[a[0]] = 0;
	for(int i = 1; i < n; i++){
		mapeo[a[i]] = mapeo[a[i - 1]] + (!(s[a[i]] == s[a[i - 1]]));
	}
	int len = 1;
	vector<int> head(n);
	vector<int> new_mapeo(n);
	vector<int> sorted_by_second(n);
	while(len < n){
		int at = 0;
		for(int i = 0; i < n; i++){
			sorted_by_second[at++] = (a[i] - len + n) % n;
		}
		for(int i = n - 1; i >= 0; i--){
			head[mapeo[a[i]]] = i;
		}
		for(int i = 0; i < n; i++){
			int x = sorted_by_second[i];
			a[head[mapeo[x]]++] = x;
		}
		new_mapeo[a[0]] = 0;
		for(int i = 1; i < n; i++){
			if(mapeo[a[i]] != mapeo[a[i - 1]]){
				new_mapeo[a[i]] = new_mapeo[a[i - 1]] + 1;
			}
			else{
				int pre = mapeo[(a[i] + len) % n];
				int cur = mapeo[(a[i - 1] + len) % n];
				new_mapeo[a[i]] = new_mapeo[a[i - 1]] + (pre != cur);
			}
		}
		swap(mapeo, new_mapeo);
		len <<= 1;
	}
	return a;
}
```

Esta versión mejora la constante ligeramente y además usa solo una función para obtener el suffix array.

#### Problemas para implementar

- [Suffix Array](https://judge.yosupo.jp/problem/suffixarray)
- [Minimal Shift](https://acmp.ru/asp/do/index.asp?main=task&id_course=2&id_section=18&id_topic=43&id_problem=284)

### Aplicaciones del Suffix Array

Ya que sabemos construir un suffix array, podemos intentar ver algunas aplicaciones interesantes:

#### Buscar un substring y la cantidad de ocurrencias

Podemos aprovechar el hecho de que el suffix array ordena los sufijos lexicográficamente para obtener la cantidad de ocurrencias de una cadena $t$.

Dado que los mapeos son cíclicos, deberemos agregar el caracter separador $\#$ que tenga un mapeo menor que cualquier caracter de $\Sigma$ al final de $s$ y así obtendremos el ordenamiento de los sufijos ignorando el elemento $a_{0}$, pues este representará al caracter $\#$.

Entonces, para buscar a $t$ (sea $|t| = m$), podemos usar binary search y una comparación lineal en **solo los primeros $m$ caracteres**.

```C++
bool check(int start){
	for(int i = 0; i < m; i++){
		if(start + i >= n) return false;
		if(s[start + i] != t[i]) return s[start + i] < t[i];
	}
	return false;
}

bool find(vector<int> &a){
	int lo = 1, hi = n - 1;
	while(lo < hi){
		int mi = lo + (hi - lo) / 2;
		if(check(a[mi])) lo = mi + 1;
		else hi = mi;
	}
	int start = a[lo];
	for(int i = 0; i < m; i++){
		if(start + i >= n or s[start + i] != t[i]) return false;
	}
	return true;
}
```

La complejidad del algoritmo anterior es de $O(m\log{n})$. Se puede extender la idea a hallar la primera y la última posición que contienen a $t$ y con eso se pueden hallar la cantidad de ocurrencias.

### LCP array

El LCP array es una manera de aprovechar información relacionada con el suffix array. Es un arreglo de tamaño $n - 1$ definido como:

$$ lcp_{i} = |LCP(a_{i}, a_{i + 1})| $$

Donde $LCP(i, j)$ es el *Longest Common Prefix* de los sufijos que empiezan en las posiciones $i$ y $j$, que por su traducción es el prefijo en común más largo entre esos dos substrings.

#### Construcción en tiempo lineal

Para construir el LCP array en tiempo lineal realizaremos la siguiente observación:

Consideremos que vamos a hallar el valor $lcp_{i}$, entonces estaremos analizando los sufijos $a_{i}$ y $a_{i + 1}$. Sea su lcp igual a $k$; entonces, si le quitamos la primera letra a los dos sufijos estaremos trabajando con los sufijos $a_{i} + 1$ y $a_{i + 1} + 1$. Dado que $a_{i}$ está antes que $a_{i + 1}$ en el suffix array, es evidente que $a_{i} + 1$ estará antes que $a_{i + 1} + 1$, por lo tanto sus primeros $\max{\{0, k - 1\}}$ caracteres serán iguales.

Algo que es importante notar es que pueden haber más caracteres en común, así que deberemos realizar un bucle `while` hasta que ya no coincidan. Luego de este procedimiento, restaremos $1$ al $k$ en representación de quitar la primera letra. 

Ahora, para que lo anterior funcione, deberemos trabajar con los sufijos en orden de tamaño decreciente y además mantener un arreglo $rank$ que nos dé la posición del sufijo $i$ en el suffix array.

```C++
vector<int> lcp_construction(vector<int> &s, vector<int> &a){
	int n = s.size();
	vector<int> rank(n, 0);
	for(int i = 0; i < n; i++){
		rank[a[i]] = i;
	}
	int k = 0;
	vector<int> lcp(n - 1, 0);
	for(int i = 0; i < n; i++){
		if(rank[i] == n - 1){
			k = 0;
			continue;
		}
		int j = a[rank[i] + 1];
		while(i + k < n and j + k < n and s[i + k] == s[j + k]) k += 1;
		lcp[rank[i]] = k;
		if(k > 0) k -= 1;
	}
	return lcp;
}
```

Dado que reducimos $k$ a lo mucho $n - 1$ veces (todas excepto cuando $rank[i] = n - 1$) y que en todo momento se debe cumplir que $k \leq n$, la cantidad de iteraciones del while (o, en otras palabras, la cantidad de veces que aumenta el valor de $k$) es $O(n)$.

### Aplicaciones con el LCP array

#### Obtener el LCP de dos sufijos

Para obtener el LCP entre dos sufijos $a_{i}$ y $a_{j}$ con $i \leq j$, nos basta con obtener el mínimo entre todos los valores $lcp_{k}$ con $k = i, \ldots, j - 1$. Esto quiere decir que podemos permitirnos realizar varias queries sobre el LCP entre dos sufijos si construimos una estructura para RMQ.

$$ LCP(a_{i}, a_{j}) = \min\limits_{i \leq k < j}{\{lcp_{k}\}} $$

#### Encontrar la cantidad de substrings diferentes

Para encontrar la cantidad de substrings diferentes de $s$, podemos construir su suffix array y lcp array y notar lo siguiente:

Cada substring es un prefijo de un sufijo, entonces nos basta con agregar todos los substrings que aporta el sufijo $a_{i}$ y restarle todos los substrings que tiene en común con el sufijo $a_{i - 1}$ (Para $i = 1$, agregamos todos de manera directa). Se puede probar por inducción que la respuesta final será la cantidad de substrings diferentes:

$$ \text{Substrings diferentes} = \sum\limits_{i = 1}^{n}i - \sum\limits_{i = 0}^{n - 2}lcp_{i} = \frac{n(n + 1)}{2} - \sum\limits_{i = 0}^{n - 2}lcp_{i} $$

Por lo tanto, podemos obtener la respuesta en $O(n)$ con un preprocesamiento de $O(n\log{n})$.

### Problemas para practicar

- [GATTACA](https://onlinejudge.org/index.php?option=com_onlinejudge&Itemid=8&page=show_problem&problem=2507)
- [CF Edu](https://codeforces.com/edu/course/2/lesson/2)